In [ ]:
# -*- coding: utf-8 -*-
"""
QAP Negative Binomial GLM for OD flows in Python (ArcGIS Pro) with:
- Optional exposure offset (dyadic field; used as log(offset) in the model)
- Directed or Undirected handling (Undirected collapses i<->j pairs)

Script tool parameters (ORDER MATTERS):
  0 in_table         (GDB table path)
  1 source_field     (e.g., "SourceID")
  2 target_field     (e.g., "TargetID")
  3 dep_field        (count-dependent variable, e.g., "Weight")
  4 predictor_fields (multi-value or semicolon-separated list)
  5 exposure_field   (optional; blank to omit; e.g., "Exposure")
  6 reps             (number of QAP permutations, e.g., 5000)
  7 directed         ("TRUE"/"FALSE")
  8 out_coef_csv     (output coefficients CSV path)
  9 out_pred_csv     (output predictions CSV path)

Outputs:
- Coef CSV: Predictor, Estimate, PermP
  (Estimates are NB-GLM coefficients on log scale; exp(Estimate)=IRR)
- Predictions CSV: SourceID, TargetID, Observed, Predicted, Residual

Requires: statsmodels (install in ArcGIS Pro env, e.g., conda install -n arcgispro-py3 statsmodels)
"""

import arcpy
import numpy as np
import pandas as pd
import os

# Try to import statsmodels; fail with a clear message if missing.
try:
    import statsmodels.api as sm
except Exception:
    arcpy.AddError(
        "This tool requires the 'statsmodels' Python package.\n"
        "Install with: conda install -n arcgispro-py3 statsmodels"
    )
    raise


def table_to_df(table, fields):
    arr = arcpy.da.TableToNumPyArray(table, fields, skip_nulls=False)
    if hasattr(arr, "mask"):
        df = pd.DataFrame(arr.filled(np.nan))
    else:
        df = pd.DataFrame(arr)
    return df


def build_matrix(nodes, src_vec, tgt_vec, val_vec):
    n = len(nodes)
    idx_map = {int(nid): i for i, nid in enumerate(nodes)}
    I = np.array([idx_map[int(s)] for s in src_vec], dtype=int)
    J = np.array([idx_map[int(t)] for t in tgt_vec], dtype=int)
    M = np.full((n, n), np.nan, dtype=float)
    M[I, J] = val_vec.astype(float)
    np.fill_diagonal(M, np.nan)  # no self-ties
    return M, I, J


def sym_mean(A):
    """Symmetrize by nan-mean across ij and ji (keeps NaN if both are NaN)."""
    AT = A.T
    both_nan = np.isnan(A) & np.isnan(AT)
    S = np.nanmean(np.dstack([A, AT]), axis=2)
    S[both_nan] = np.nan
    np.fill_diagonal(S, np.nan)
    return S


def undirected_upper_from_mats(Y, Xlist, E=None, sum_counts=True):
    """
    Build undirected vectors (upper triangle i<j):
      - Counts: sum across directions if sum_counts=True, else mean across directions
      - Predictors and Exposure: nan-mean across directions (symmetric)
    Returns y_vec, X_mat, E_vec, idx_i, idx_j
    """
    n = Y.shape[0]
    iu, ju = np.triu_indices(n, k=1)

    # Counts: sum across directions (treat missing as 0, but keep NaN if both missing)
    Y0 = np.nan_to_num(Y, nan=0.0)
    YT0 = np.nan_to_num(Y.T, nan=0.0)
    Ysum = Y0 + YT0
    both_nan = np.isnan(Y) & np.isnan(Y.T)
    y_vec = Ysum[iu, ju]
    y_vec = np.where(both_nan[iu, ju], np.nan, y_vec)
    if not sum_counts:
        # Replace sum with nan-mean across directions
        y_mean = np.nanmean(np.dstack([Y, Y.T]), axis=2)
        y_vec = y_mean[iu, ju]

    # Predictors: nan-mean across directions
    X_cols = []
    for Xk in Xlist:
        Xs = np.nanmean(np.dstack([Xk, Xk.T]), axis=2)
        X_cols.append(Xs[iu, ju])
    X_mat = np.column_stack(X_cols) if X_cols else np.empty((len(y_vec), 0))

    # Exposure: nan-mean across directions (avoids doubling when symmetric)
    if E is not None:
        Es = np.nanmean(np.dstack([E, E.T]), axis=2)
        E_vec = Es[iu, ju]
    else:
        E_vec = None

    return y_vec, X_mat, E_vec, iu, ju


def _alpha_moments(y, mu, eps=1e-12):
    """
    Method-of-moments NB2 alpha estimate:
      Var(y) = mu + alpha * mu^2  =>  alpha = sum((y-mu)^2 - y) / sum(mu^2)
    Clipped to be >= eps.
    """
    y = np.asarray(y, dtype=float)
    mu = np.asarray(mu, dtype=float)
    num = float(np.sum((y - mu) ** 2 - y))
    den = float(np.sum(mu ** 2))
    if den <= 0:
        return eps
    alpha = num / den
    return float(max(alpha, eps))


def fit_nb_glm(y, X, offset=None, max_alpha_iter=10, alpha_tol=1e-3):
    """
    Fit a Negative Binomial GLM with log link using statsmodels.
    Alpha is estimated iteratively by method-of-moments.

    y: 1D array of non-negative counts.
    X: 2D array of predictors (without intercept).
    offset: 1D array on linear predictor scale (i.e., log(exposure)); or None.

    Returns:
      beta (including intercept),
      mu (fitted means),
      residuals (y - mu),
      AIC, log-likelihood, alpha
    """
    y = np.asarray(y, dtype=float)
    X = np.asarray(X, dtype=float)
    X1 = np.column_stack([np.ones(len(y), dtype=float), X])  # add intercept
    off = None if offset is None else np.asarray(offset, dtype=float)

    # Start with Poisson to initialize mu and alpha
    pois = sm.GLM(y, X1, family=sm.families.Poisson(), offset=off).fit(maxiter=200, tol=1e-8)
    mu = pois.fittedvalues.astype(float)
    alpha = _alpha_moments(y, mu)

    # Iterate NB with updated alpha
    res_nb = None
    for _ in range(max_alpha_iter):
        fam = sm.families.NegativeBinomial(alpha=alpha)
        res_nb = sm.GLM(y, X1, family=fam, offset=off).fit(maxiter=200, tol=1e-8)
        mu_new = res_nb.fittedvalues.astype(float)
        alpha_new = _alpha_moments(y, mu_new)
        if abs(alpha_new - alpha) / (alpha + 1e-12) < alpha_tol:
            mu = mu_new
            alpha = alpha_new
            break
        mu = mu_new
        alpha = alpha_new

    if res_nb is None:
        # Fallback to Poisson
        beta = pois.params.astype(float)
        mu = pois.fittedvalues.astype(float)
        resid_resp = (y - mu).astype(float)
        aic = float(pois.aic)
        llf = float(pois.llf)
        alpha = 0.0
        return beta, mu, resid_resp, aic, llf, alpha

    beta = res_nb.params.astype(float)
    mu = res_nb.fittedvalues.astype(float)
    resid_resp = (y - mu).astype(float)
    aic = float(res_nb.aic)
    llf = float(res_nb.llf)
    return beta, mu, resid_resp, aic, llf, float(alpha)


def qap_glm_nb(df, src_field, tgt_field, dep_field, pred_fields,
               reps=5000, directed=True, exposure_field=None, seed=123):
    # Clean types and ensure non-negative counts
    df = df.dropna(subset=[src_field, tgt_field, dep_field] + pred_fields).copy()
    df[src_field] = df[src_field].astype(int)
    df[tgt_field] = df[tgt_field].astype(int)
    for f in pred_fields:
        df[f] = df[f].astype(float)
    df[dep_field] = np.maximum(0.0, df[dep_field].astype(float))

    # If exposure provided, clean it (must be > 0); small floor avoids log(0)
    if exposure_field and exposure_field.strip():
        df[exposure_field] = df[exposure_field].astype(float)
        df[exposure_field] = np.where(df[exposure_field] <= 0, 1e-12, df[exposure_field])
        use_offset = True
    else:
        use_offset = False

    # Node set
    nodes = sorted(set(df[src_field]).union(set(df[tgt_field])))
    nodes = [int(n) for n in nodes]

    # Build matrices
    Y, I, J = build_matrix(
        nodes, df[src_field].values, df[tgt_field].values, df[dep_field].values
    )
    Xmats = []
    for f in pred_fields:
        Xk, _, _ = build_matrix(
            nodes, df[src_field].values, df[tgt_field].values, df[f].values
        )
        Xmats.append(Xk)
    Emat = None
    if use_offset:
        Emat, _, _ = build_matrix(
            nodes, df[src_field].values, df[tgt_field].values, df[exposure_field].values
        )

    # Observed vectors
    if directed:
        # Use exactly the rows in the table
        y_obs = Y[I, J]
        X_obs = np.column_stack([Xk[I, J] for Xk in Xmats]) if Xmats else np.empty((len(y_obs), 0))
        mask = (~np.isnan(y_obs)) & (np.all(~np.isnan(X_obs), axis=1) if X_obs.size else True)
        y_obs = y_obs[mask].astype(float)
        X_obs = X_obs[mask, :].astype(float) if X_obs.size else np.empty((mask.sum(), 0))
        src_obs = df[src_field].values[mask].astype(int)
        tgt_obs = df[tgt_field].values[mask].astype(int)
        if use_offset:
            E_obs = Emat[I, J][mask].astype(float)
            E_obs = np.where(E_obs <= 0, 1e-12, E_obs)
            off_obs = np.log(E_obs)
        else:
            off_obs = None
    else:
        # Undirected: collapse to unique unordered pairs (i<j)
        # Counts are summed across directions; predictors and exposure averaged
        y_vec, X_mat, E_vec, iu, ju = undirected_upper_from_mats(
            Y, Xmats, E=Emat, sum_counts=True
        )
        valid = ~np.isnan(y_vec)
        if X_mat.size:
            valid &= np.all(~np.isnan(X_mat), axis=1)
        y_obs = y_vec[valid].astype(float)
        X_obs = X_mat[valid, :].astype(float) if X_mat.size else np.empty((valid.sum(), 0))
        src_obs = np.array([nodes[i] for i in iu[valid]], dtype=int)
        tgt_obs = np.array([nodes[j] for j in ju[valid]], dtype=int)
        if use_offset:
            E_obs = E_vec[valid].astype(float)
            E_obs = np.where(E_obs <= 0, 1e-12, E_obs)
            off_obs = np.log(E_obs)
        else:
            off_obs = None

    if y_obs.size == 0:
        raise ValueError("No valid rows to model after cleaning.")
    if y_obs.size <= X_obs.shape[1]:
        raise ValueError("Not enough observations to estimate all parameters.")

    # Fit observed NB-GLM with optional offset
    beta_obs, mu_obs, resid_obs, aic_obs, llf_obs, alpha_obs = fit_nb_glm(
        y_obs, X_obs, offset=off_obs
    )
    k = beta_obs.size
    n = y_obs.size
    aicc_obs = aic_obs + (2.0 * k * (k + 1)) / (n - k - 1) if (n - k - 1) > 0 else float("nan")

    # QAP permutations
    rng = np.random.default_rng(seed)
    n_nodes = len(nodes)
    B = np.full((reps, k), np.nan, dtype=float)

    for r in range(reps):
        perm = rng.permutation(n_nodes)
        Yp = Y[perm][:, perm]
        Xps = [Xk[perm][:, perm] for Xk in Xmats]
        Ep = Emat[perm][:, perm] if use_offset else None

        try:
            if directed:
                yp = Yp[I, J][mask]
                if Xmats:
                    Xp = np.column_stack([Xpk[I, J][mask] for Xpk in Xps])
                else:
                    Xp = np.empty((yp.size, 0))
                if use_offset:
                    Epv = Ep[I, J][mask]
                    Epv = np.where(np.asarray(Epv) <= 0, 1e-12, Epv)
                    offv = np.log(Epv)
                else:
                    offv = None
                valid = (~np.isnan(yp)) & (np.all(~np.isnan(Xp), axis=1) if Xp.size else True)
                yp = yp[valid]
                Xp = Xp[valid, :] if Xp.size else Xp
                offv = offv[valid] if offv is not None else None
            else:
                yv, Xv, Ev, iu_p, ju_p = undirected_upper_from_mats(
                    Yp, Xps, E=Ep, sum_counts=True
                )
                valid = ~np.isnan(yv)
                if Xv.size:
                    valid &= np.all(~np.isnan(Xv), axis=1)
                yp = yv[valid]
                Xp = Xv[valid, :] if Xv.size else Xv
                if use_offset:
                    Ev = np.where(np.asarray(Ev) <= 0, 1e-12, Ev)
                    offv = np.log(Ev[valid])
                else:
                    offv = None

            if yp.size <= k - 1:
                continue

            bp, _, _, _, _, _ = fit_nb_glm(yp.astype(float), Xp.astype(float), offset=offv)
            if bp.size == k:
                B[r, :] = bp
        except Exception:
            # Non-convergence or numerical issue; skip this permutation
            continue

    # Two-sided permutation p-values
    pvals = []
    for j in range(k):
        bj = beta_obs[j]
        perm_vals = B[:, j]
        perm_vals = perm_vals[~np.isnan(perm_vals)]
        if perm_vals.size == 0:
            p = np.nan
        else:
            p = (np.sum(np.abs(perm_vals) >= np.abs(bj)) + 1.0) / (perm_vals.size + 1.0)
        pvals.append(p)

    return (
        beta_obs,
        pvals,
        mu_obs,
        resid_obs,
        src_obs,
        tgt_obs,
        aic_obs,
        aicc_obs,
        llf_obs,
        alpha_obs,
    )


def main():
    in_table = arcpy.GetParameterAsText(0)
    src_field = arcpy.GetParameterAsText(1)
    tgt_field = arcpy.GetParameterAsText(2)
    dep_field = arcpy.GetParameterAsText(3)
    preds_param = arcpy.GetParameterAsText(4)
    exposure_field = arcpy.GetParameterAsText(5)  # may be empty
    reps = int(arcpy.GetParameterAsText(6))
    directed_str = arcpy.GetParameterAsText(7)
    out_coef_csv = arcpy.GetParameterAsText(8)
    out_pred_csv = arcpy.GetParameterAsText(9)

    directed = True
    if isinstance(directed_str, str):
        directed = directed_str.strip().upper() in ("TRUE", "T", "YES", "Y", "1")

    # Predictor fields may arrive as list or semicolon string
    if isinstance(preds_param, list):
        pred_fields = preds_param
    else:
        pred_fields = [p for p in preds_param.split(";") if p]

    fields = [src_field, tgt_field, dep_field] + pred_fields
    if exposure_field and exposure_field.strip():
        fields.append(exposure_field)

    df = table_to_df(in_table, fields)

    (
        beta,
        pvals,
        mu,
        resid,
        src_obs,
        tgt_obs,
        aic,
        aicc,
        llf,
        alpha,
    ) = qap_glm_nb(
        df,
        src_field,
        tgt_field,
        dep_field,
        pred_fields,
        reps=reps,
        directed=directed,
        exposure_field=(exposure_field if exposure_field and exposure_field.strip() else None),
    )

    # Coefficients CSV
    coef_names = ["Intercept"] + pred_fields
    coef_df = pd.DataFrame({"Predictor": coef_names, "Estimate": beta, "PermP": pvals})
    os.makedirs(os.path.dirname(out_coef_csv), exist_ok=True)
    coef_df.to_csv(out_coef_csv, index=False)

    # Predictions CSV
    pred_df = pd.DataFrame(
        {
            "SourceID": src_obs.astype(int),
            "TargetID": tgt_obs.astype(int),
            "Observed": (mu + resid).astype(float),
            "Predicted": mu.astype(float),
            "Residual": resid.astype(float),
        }
    )
    os.makedirs(os.path.dirname(out_pred_csv), exist_ok=True)
    pred_df.to_csv(out_pred_csv, index=False)

    arcpy.AddMessage("QAP Negative Binomial GLM completed.")
    arcpy.AddMessage(f"AIC: {aic:.6f}")
    arcpy.AddMessage(f"AICc: {aicc:.6f}")
    arcpy.AddMessage(f"Log-likelihood: {llf:.6f}")
    arcpy.AddMessage(f"Estimated overdispersion (alpha): {alpha:.6g}")
    arcpy.AddMessage(f"Directed: {directed}")
    if exposure_field and exposure_field.strip():
        arcpy.AddMessage(f"Offset field used (log scale): {exposure_field}")
    else:
        arcpy.AddMessage("No offset field used.")
    arcpy.AddMessage(f"Coefficients CSV: {out_coef_csv}")
    arcpy.AddMessage(f"Predictions CSV: {out_pred_csv}")


if __name__ == "__main__":
    main()